In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from pyBedGraph import BedGraph
from pybedtools import BedTool
import itertools

In [2]:
def read_left(directory, file_name):
    with open(directory + file_name) as f:
        counts = {}
        for line in f:
            tmp = line.strip().split("\t")
            if tmp[0] != 'left_count':
                counts[tmp[5]] = [int(tmp[0])+int(tmp[1]), int(tmp[2])]
    return counts

In [3]:
def read_right(directory, file_name):
    with open(directory + file_name) as f:
        counts = {}
        for line in f:
            tmp = line.strip().split("\t")
            if tmp[0] != 'left_count':
                counts[tmp[5]] = [int(tmp[3])+int(tmp[4]), int(tmp[2])]
    return counts

In [4]:
def read_loops(directory, file_name):
    with open(directory + file_name) as f:
        loops = []
        for line in f:
            tmp = line.strip().split("\t")
            tmp[1] = int(tmp[1])
            tmp[2] = int(tmp[2])
            tmp[4] = int(tmp[4])
            tmp[5] = int(tmp[5])
            loops.append(tmp)
    return loops

In [5]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [6]:
directory='/Users/kimm/Desktop/GM12878_files/'
loop_file='LHG0052H.e500.clusters.cis.bothanchint_G250.PETcnt_G9.motifannot.sorted.loops'
left_file='mid_left_gem_count_matrix_convergent_loops.txt'
right_file='mid_right_gem_count_matrix_convergent_loops.txt'

In [7]:
loops = read_loops(directory, loop_file)
leftcnt = read_left(directory, left_file)
rightcnt = read_right(directory, right_file)

In [21]:
conv_loops = [x for x in loops if x[15]=='+' and x[20]=='-']

In [22]:
def motifoverlap_left(motif, loop_id, loops):
    overlapped = []
    for x in loops:
        rightmotif = [x[17], x[18], x[19], x[20]]
        if motif == rightmotif and x[11] != loop_id:
            overlapped.append(x)
    return overlapped

In [23]:
def motifoverlap_right(motif, loop_id, loops):
    overlapped = []
    for x in loops:
        leftmotif = [x[12], x[13], x[14], x[15]]
        if motif == leftmotif and x[11] != loop_id:
            overlapped.append(x)
    return overlapped

In [24]:
len(conv_loops)

6385

In [90]:
left_kept = []
left_removed = []
ratiolist = []
middlelist = []
spanlist = []
overlaplist = []
for x in conv_loops:
    span = x[4]-x[2]
    spanlist.append(span)
    left = leftcnt[x[11]][0]
    middle = leftcnt[x[11]][1]
    ratiolist.append((middle+1)/(left+1))
    middlelist.append(middle)
    leftmotif = [x[12], x[13], x[14], x[15]]
    overlapped = motifoverlap_left(leftmotif, x[11], loops)
    overlaplist.append(len(overlapped))
    if span > 150000 and middle > 50: 
        if len(overlapped) == 0:
            left_kept.append([x[11], left, middle])
        else:
            left_removed.append([x[11], left, middle])
    else:
        left_removed.append([x[11], left, middle])

In [236]:
write_result(directory, left_kept, "convergent_loops_filtered_span_G150kb_middle_G35comp_nooverlap_left.bed")

In [86]:
right_kept = []
right_removed = []
for x in conv_loops:
    span = x[4]-x[2]
    right = rightcnt[x[11]][0]
    middle = rightcnt[x[11]][1]
    rightmotif = [x[17], x[18], x[19], x[20]]
    overlapped = motifoverlap_right(rightmotif, x[11], loops)
    if span > 150000 and middle > 50: 
        if len(overlapped) == 0:
            right_kept.append([x[11], right, middle])
        else:
            right_removed.append([x[11], right, middle])
    else:
        right_removed.append([x[11], right, middle])

In [292]:
write_result(directory, right_kept, "convergent_loops_filtered_span_G150kb_middle_G50comp_nooverlap_right.txt")